In [24]:
import os
from pprint import pprint

def count_relations(folder_path):
    # Create a dictionary to store the counts
    counts = {}
    # Loop through all subdirectories
    for subdir, _, files in os.walk(folder_path):
        # print(subdir)
        # Extract the code from the subdirectory name
        full_folder_name = os.path.basename(subdir).split('.')
        code = full_folder_name[0]
        # code = full_folder_name[1]
        # print(code)
        if len(full_folder_name) == 1:
            continue
        # Skip any subdirectories that don't contain a .conll file
        # if not any(f.endswith('.conll') for f in files):
        #     continue

        # print(full_folder_name[1])


        # Read the CURATION_USER.tsv file, if it exists
        user_file = os.path.join(subdir, 'CURATION_USER.tsv')
        # print("user_file ", os.path.exists(user_file), user_file)
        if not os.path.exists(user_file):
            continue
        

        with open(user_file, 'r', encoding='utf-8') as f:
            # Initialize the counts for this code
            code_counts = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0, "PERSONAL - SOCIAL": 0}
            # Loop through each line in the file
            for line in f:
                # Split the line into columns
                # columns = line.strip().split('\t')
                # # Extract the relation type from the third column
                # relation = columns[2]
                # Increment the count for this relation type, if it's one of the three we're counting
                for relation , number in code_counts.items():
                    n = line.count(relation)
                    code_counts[relation] += n
            # Store the counts for this code in the main dictionary
            counts[code] = code_counts
    return counts
dev_data_raw = "/Users/n2t2k/Documents/Studying/Master/Thesis/InProgress/Coding/ORIGIN_RUN_ALL_edge-oriented-graph-master-studying/dataProcessingOfficialCleaned/VLSP2020_RE_dev_org_use_TRAIN_as_DEV"
dev_raw_dict = count_relations(dev_data_raw)

# pprint(dev_raw_dict)

total_counts_raw = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0, "PERSONAL - SOCIAL": 0}
for rel in total_counts_raw:
    # print(rel)
    # code_counts[rel] = sum(item[rel] for item in dev_raw_dict)
    total_counts_raw[rel] = sum(dev_raw_dict[item][rel] for item in dev_raw_dict)

print(total_counts_raw)

{'LOCATED': 346, 'AFFILIATION': 518, 'PART - WHOLE': 0, 'PERSONAL - SOCIAL': 98}


In [25]:
import os

def count_relations_in_folder(folder_path, relation_types):
    counts = {}
    for filename in os.listdir(folder_path):
        code_counts = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0, "PERSONAL - SOCIAL": 0}
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            code = filename.split('.')[0]
            # print(file_path)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    for relation , number in code_counts.items():
                        n = line.count(relation)
                        code_counts[relation] += n
            counts[code] = code_counts

    return counts

dev_data_pubtabtor =  "/Users/n2t2k/Documents/Studying/Master/Thesis/InProgress/Coding/ORIGIN_RUN_ALL_edge-oriented-graph-master-studying/dataProcessingOfficialCleaned/dev_processed/processed_txt"
rel = ['LOCATED', 'AFFILIATION', 'PART - WHOLE',  "PERSONAL - SOCIAL"] 
dev_data_pubtabtor_dict = count_relations_in_folder(dev_data_pubtabtor, rel)
# pprint(dev_data_pubtabtor_dict)


total_counts_pubtabtor = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0, "PERSONAL - SOCIAL": 0}
for rel in total_counts_pubtabtor:
    # print(rel)
    # code_counts[rel] = sum(item[rel] for item in dev_raw_dict)
    total_counts_pubtabtor[rel] = sum(
            (dev_raw_dict[item][rel] if dev_raw_dict.get(item, None) else 0)
            for item in dev_data_pubtabtor_dict)

print(total_counts_pubtabtor)

{'LOCATED': 314, 'AFFILIATION': 407, 'PART - WHOLE': 0, 'PERSONAL - SOCIAL': 68}


In [26]:
def count_types_final(filename):
    type_count = {}
    with open(filename, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            code = parts[0]
            # if code not in type_count:
            type_count[code] = {'LOCATED': 0, 'AFFILIATION': 0, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': 0}
            # for i in range(1, len(parts)):
            #     if 'NON-CROSS' in parts[i]:
            #         type_parts = parts[i].split('\t')
            #         type_name = type_parts[0]
            #         if type_name in type_count[code]:
            #             type_count[code][type_name] += 1

            for relation , number in type_count[code].items():
                type_count[code][relation] = line.count(relation)

    return type_count

dev_data_final_dict_path = \
    "/Users/n2t2k/Documents/Studying/Master/Thesis/InProgress/Coding/ORIGIN_RUN_ALL_edge-oriented-graph-master-studying/dataProcessingOfficialCleaned/dev_processed/split_sentence_underthesea/docs/common_info_embedd_files/vlsp.data"
dev_data_final_dict = count_types_final(dev_data_final_dict_path)
# pprint(dev_data_final_dict)

total_countfinal_dict = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0, "PERSONAL - SOCIAL": 0}
for rel in total_countfinal_dict:
    print(rel)
    # total_countfinal_dict[rel] = sum(
    #             dev_data_final_dict[item][rel] for item in dev_data_final_dict
    #             )
    total_countfinal_dict[rel] = sum(
            (dev_data_final_dict[item][rel] if dev_data_final_dict[item].get(rel, None) else 0)
            for item in dev_data_final_dict)


print("total_counts_pubtabtor ", total_counts_pubtabtor)
print("total_countfinal_dict ", total_countfinal_dict)


dev_data_dict_subtract_pubtator_final = subtract_dicts(dev_data_final_dict,dev_data_pubtabtor_dict)
print("dev_data_dict_subtract >>> ")
# pprint(dev_data_dict_subtract)

i=1
for code, _count in dev_data_dict_subtract_pubtator_final.items():
    if dev_data_pubtabtor_dict.get(code, None) and dev_data_final_dict.get(code, None):
        if _count['AFFILIATION'] or _count['LOCATED'] or _count['PERSONAL - SOCIAL']:
            i+=1
            print(f"{i}: {code}, {_count}")

LOCATED
AFFILIATION
PART - WHOLE
PERSONAL - SOCIAL
total_counts_pubtabtor  {'LOCATED': 314, 'AFFILIATION': 407, 'PART - WHOLE': 0, 'PERSONAL - SOCIAL': 68}
total_countfinal_dict  {'LOCATED': 445, 'AFFILIATION': 656, 'PART - WHOLE': 0, 'PERSONAL - SOCIAL': 91}
dev_data_dict_subtract >>> 
2: 23351519, {'LOCATED': 0, 'AFFILIATION': 0, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': -1}
3: 23352662, {'LOCATED': 0, 'AFFILIATION': -1, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': 0}
4: 23353931, {'LOCATED': 0, 'AFFILIATION': -1, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': 0}
5: 23354253, {'LOCATED': 0, 'AFFILIATION': -1, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': 0}
6: 23355001, {'LOCATED': -1, 'AFFILIATION': 0, 'PART – WHOLE': 0, 'PERSONAL - SOCIAL': 0}


In [27]:
def subtract_dicts(d1, d2):
    result = {}
    for code in d1:
        if code not in d2:
            result[code] = d1[code]
        else:
            result[code] = {}
            for t in d1[code]:
                result[code][t] = d1[code][t] - d2[code].get(t, 0)
    return result

dev_data_dict_subtract = subtract_dicts(dev_data_pubtabtor_dict, dev_raw_dict)
print("dev_data_dict_subtract >>> ")
# pprint(dev_data_dict_subtract)
i = 0
for code, _count in dev_data_dict_subtract.items():
    if dev_data_pubtabtor_dict.get(code, None) and dev_raw_dict.get(code, None):
        if _count['AFFILIATION'] or _count['LOCATED'] or _count['PART - WHOLE'] or _count["PERSONAL - SOCIAL"]:
            i+=1
            print(f"{i}: {code}, {_count}")


dev_data_dict_subtract >>> 
1: 23352640, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 1, 'PERSONAL - SOCIAL': 0}
2: 23352597, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 7, 'PERSONAL - SOCIAL': 0}
3: 23352443, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 11, 'PERSONAL - SOCIAL': 0}
4: 23352396, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 1, 'PERSONAL - SOCIAL': 0}
5: 23352401, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 4, 'PERSONAL - SOCIAL': 0}
6: 23352292, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 1, 'PERSONAL - SOCIAL': 0}
7: 23352079, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 4, 'PERSONAL - SOCIAL': 0}
8: 23352415, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 2, 'PERSONAL - SOCIAL': 0}
9: 23352623, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 22, 'PERSONAL - SOCIAL': 0}
10: 23352373, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 7, 'PERSONAL - SOCIAL': 0}
11: 23352572, {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 1, 'PERSONAL - S